In [1]:
import os
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble, lda,
                     random_projection)

from skimage.feature import hog
from skimage import color
from scipy import misc
from skimage.transform import resize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
src_path, image_type = "/Users/myazdaniUCSD/Desktop/PNAR/", ".jpg"
#src_path = "/Users/myazdaniUCSD/Documents/paintings/ROTHKO_MONDRIAN/Rothko_Mondrian_Complete image sets/Rothko_images/"
 
image_paths = []  
for root, dirs, files in os.walk(src_path):
    image_paths.extend([os.path.join(root, f) for f in files if f.endswith(image_type)])

In [3]:
def return_HOG(img_path):
    img = misc.imread(img_path)
    hogs = []
    for i in range(3):
        img_c = misc.imresize(img[:,:,i], (80,100))
        hog_c = hog(img_c, orientations=16, pixels_per_cell=(2, 2),cells_per_block=(1, 1), visualise=False)
        hogs.extend(hog_c)
    return array(hogs)

# def return_HOG(img_path):
#     img = misc.imread(img_path)
#     new_img = misc.imresize(color.rgb2gray(img), (80,100))
#     return hog(new_img, orientations=16, pixels_per_cell=(2, 2),cells_per_block=(1, 1), visualise=False)

In [4]:
image_features = [return_HOG(image_path) for image_path in image_paths]

# Reduce dimensions and apply clustering

In [5]:
pca = PCA(n_components=50)
X = pca.fit_transform(array(image_features))

In [6]:
num_clusters = 100
kmeans = KMeans(n_clusters = num_clusters, n_init = 100)
#kmeans = KMeans(n_clusters = num_clusters)
clusters = kmeans.fit_predict(X)

#Apply 1-D tsne on cluster centers

In [7]:
tsne = manifold.TSNE(n_components=1, init='pca', random_state=0)
cluster_tsne = tsne.fit_transform(kmeans.cluster_centers_)

In [8]:
cluster_dict = {}
cluster_key = argsort(cluster_tsne[:,0])

In [9]:
cluster_key = argsort(cluster_tsne[:,0])
ordered_clusters = [] 
for cluster in clusters:
    ordered_clusters.append(cluster_key[cluster])

In [10]:
data_features = pd.DataFrame({"image.paths" : image_paths})
data_features["ordered.clusters"] = ordered_clusters
data_features["clust.dist"] = ""

#Apply tsne for each cluster

In [11]:
for cluster in range(num_clusters):
    clust_indx = squeeze(argwhere(array(ordered_clusters) == cluster))
    X_cluster = X[clust_indx,:]
    data_features.loc[clust_indx,"clust.dist"] = rint(100*tsne.fit_transform(X_cluster))

In [12]:
data_features.to_csv("../results/Rothko-KMEANStsne-HOG-color.csv", index = False, header = False)